In [1]:
# 用一个下载文件的例子来说明使用多进程和不使用多进程到底有什么差别
from random import randint
from time import time, sleep

def download_task(filename):
    print(f'开始下载{filename}')
    time_to_download = randint(5, 10)
    sleep(time_to_download)
    print(f'{filename}下载完成!耗时{time_to_download}秒')
    
def main():
    start = time()
    download_task('Python从入门到zhyua.pdf')
    download_task('Peking Hot.avi')
    end = time()
    print(f'总耗费{end - start}秒')
    
if __name__ == '__main__':
    main()

开始下载Python从入门到zhyua.pdf
Python从入门到zhyua.pdf下载完成!耗时7秒
开始下载Peking Hot.avi
Peking Hot.avi下载完成!耗时6秒
总耗费13.000598192214966秒


In [11]:
# 使用多进程的方式将两个下载任务放到不同进程中
from multiprocessing import Process
from os import getpid
from random import randint
from time import time,sleep

def download_task(filename):
    print(f'启动下载进程，进程号{getpid()}')
    print(f'开始下载{filename}...')
    time_to_download = randint(5,10)
    sleep(time_to_download)
    print('{filename}下载完成！耗费了{time_to_download}秒')
    
def main():
    start = time()
    p1 = Process(target=download_task, args=('Python从入门到住院.pdf',))
    p1.start()
    p2 = Process(target=download_task,args=('Peing Hot.avi',))
    p2.start()
    p1.join()
    p2.join()
    end = time()
    print(f'共耗费了{end-start}秒')
    
if __name__ == '__main__':
    main()

共耗费了0.10970807075500488秒


In [14]:
from multiprocessing import Process
from time import sleep

counter = 0

def sub_task(string):
    global counter
    while counter < 10:
        print(string, end='', flush = True)
        counter += 1
        sleep(0.01)
        
def main():
    Process(target=sub_task, args=('Ping',)).start()
    Process(target=sub_task,args=('pong',)).start()
    
if __name__ == '__main__':
    main()

In [15]:
# 使用多线程实现下载文件的例子
from random import randint
from threading import Thread
from time import time,sleep

def download(filename):
    print(f'开始下载{filename}')
    time_to_download = randint(5,10)
    sleep(time_to_download)
    print(f'{filename}下载完了！耗费了{time_to_download}')
    
def main():
    start = time()
    t1 = Thread(target=download, args=('Python从入门到住院.pdf',))
    t1.start()
    t2 = Thread(target=download, args=('Peking Hot.avi',))
    t2.start()
    t1.join()
    t2.join()
    end = time()
    print(f'总共花费了{end - start}秒')
    
if __name__ == '__main__':
    main()

开始下载Python从入门到住院.pdf开始下载Peking Hot.avi

Python从入门到住院.pdf下载完了！耗费了6
Peking Hot.avi下载完了！耗费了8
总共花费了8.02104663848877秒


In [18]:
# 通过继承Thread类的方式来创建自定义的线程类，然后再创建线程对象并启动线程。代码如下所示。
from random import randint
from threading import Thread
from time import time,sleep

class DownloadTask(Thread):
    def __init__(self,filename):
        super().__init__()
        self._filename = filename
    
    def run(self):
        print(f'开始下载{self._filename}')
        time_to_download = randint(5,10)
        sleep(time_to_download)
        print(f'{self._filename}下载完了！耗费了{time_to_download}秒')

def main():
    start = time()
    t1 = DownloadTask('Python从入门到zhuyuan.pdf')
    t1.start()
    t2 = DownloadTask('Peking Hot.avi')
    t2.start()
    t1.join()
    t2.join()
    end = time()
    print(f'总共花费了{end - start}秒')
    
if __name__ == '__main__':
    main()

开始下载Python从入门到zhuyuan.pdf
开始下载Peking Hot.avi
Python从入门到zhuyuan.pdf下载完了！耗费了7秒
Peking Hot.avi下载完了！耗费了9秒
总共花费了9.019232749938965秒


In [26]:
# 100个线程向同一个银行账户转账
from time import sleep
from threading import Thread

class Account:
    def __init__(self):
        self._balance = 0
    
    def deposit(self, money):
        #计算存款后的余额
        new_balance = self._balance + money
        # 模拟存款业务需要0.01秒时间
        sleep(0.01)
        # 修改账户存款
        self._balance = new_balance
        
    @property
    def balance(self):
        return self._balance
    
class AddMoneyThread(Thread):
    def __init__(self, account, money):
        super().__init__()
        self._account = account
        self._money = money
        
    def run(self):
        self._account.deposit(self._money)
        
def main():
    account = Account()
    threads = []
    # 创建100个存款的线程向同一个账户中存钱
    for _ in range(100):
        t = AddMoneyThread(account,1)
        threads.append(t)
        t.start()
        
    # 等所有存款的线程都执行完毕
    for t in threads:
        t.join()
    print(f'账户余额为{account.balance}')
    
if __name__ == '__main__':
    main()

账户余额为2


In [30]:
# 使用锁来保护临界资源账户
from time import sleep
from threading import Thread,Lock

class Account:
    def __init__(self):
        self._balance = 0
        self._lock = Lock()
        
    def deposit(self,money):
        #先获取锁才能执行后续的代码
        self._lock.acquire()
        try:
            new_balance = self._balance + money
            sleep(0.01)
            self._balance = new_balance
            
        finally:
            # 在finally中执行释放锁的操作，保证正常异常锁都能释放
            self._lock.release()
            
    @property
    def balance(self):
        return self._balance
    
class AddMoneyThread(Thread):
    def __init__(self, account, money):
        super().__init__()
        self._account = account
        self._money = money
        
    def run(self):
        self._account.deposit(self._money)
        
def main():
    account = Account()
    threads = []
    # 创建100个存款的线程向同一个账户中存钱
    for _ in range(100):
        t = AddMoneyThread(account,1)
        threads.append(t)
        t.start()
        
    # 等所有存款的线程都执行完毕
    for t in threads:
        t.join()
    print(f'账户余额为{account.balance}')
    
if __name__ == '__main__':
    main()

账户余额为100


In [31]:
import time
import tkinter
import tkinter.messagebox


def download():
    # 模拟下载任务需要花费10秒钟时间
    time.sleep(10)
    tkinter.messagebox.showinfo('提示', '下载完成!')


def show_about():
    tkinter.messagebox.showinfo('关于', '作者: 骆昊(v1.0)')


def main():
    top = tkinter.Tk()
    top.title('单线程')
    top.geometry('200x150')
    top.wm_attributes('-topmost', True)

    panel = tkinter.Frame(top)
    button1 = tkinter.Button(panel, text='下载', command=download)
    button1.pack(side='left')
    button2 = tkinter.Button(panel, text='关于', command=show_about)
    button2.pack(side='right')
    panel.pack(side='bottom')

    tkinter.mainloop()


if __name__ == '__main__':
    main()

In [32]:
#使用多线程将耗时间的任务放到一个独立的线程中执行，这样就不会因为执行耗时间的任务而阻塞了主线程
import time
import tkinter
import tkinter.messagebox
from threading import Thread

class DownloadTaskHandler(Thread):

    def run(self):
        time.sleep(10)
        tkinter.messagebox.showinfo('提示', '下载完成!')
        # 启用下载按钮
        button1.config(state=tkinter.NORMAL)
        
def download():
    # 禁用下载按钮
    button1.config(state=tkinter.DISABLED)
    # 通过daemon参数将线程设置为守护线程(主程序退出就不再保留执行)
    # 在线程中处理耗时间的下载任务
    DownloadTaskHandler(daemon=True).start()    
    
def show_about():
    tkinter.messagebox.showinfo('关于', '作者: 骆昊(v1.0)')
    
def main():
    top = tkinter.Tk()
    top.title('单线程')
    top.geometry('200x150')
    top.wm_attributes('-topmost', True)

    panel = tkinter.Frame(top)
    button1 = tkinter.Button(panel, text='下载', command=download)
    button1.pack(side='left')
    button2 = tkinter.Button(panel, text='关于', command=show_about)
    button2.pack(side='right')
    panel.pack(side='bottom')

    tkinter.mainloop()


if __name__ == '__main__':
    main()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1702, in __call__
    return self.func(*args)
  File "<ipython-input-32-aff8b82d41b3>", line 17, in download
    button1.config(state=tkinter.DISABLED)
NameError: name 'button1' is not defined


In [33]:
#完成1~100000000求和的计算密集型任务
from time import time

def main():
    total = 0
    start = time()
    num_list = [x for x in range(1,10000000)]
    for i in num_list:
        total += i
    print(total)
    end = time()
    print(f'execution time:{end - start}')
    
if __name__ == '__main__':
    main()

49999995000000
execution time:1.1130576133728027


In [34]:
def fib2(n):
    a,b = 0,1
    while a<n:
        a,b = b,a+b
        return a

fib2(10)

1